In [1]:
import pickle
import re
import warnings
import spacy
from spacy.lang.en import English
from collections import Counter, OrderedDict
import string
import swifter
import matplotlib.pyplot as plt
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd
import seaborn as sns
from fuzzywuzzy import fuzz, process
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
stop = stopwords.words('english')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
df = pd.read_csv('./data/test.csv')

In [3]:
df['prod_clean'] = df['product_title'].str.lower().apply(
    lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['prod_clean'] = df['prod_clean'].apply(
    lambda x: x.strip(string.punctuation))
df['prod_clean'] = df['prod_clean'].str.replace('\d+', '')

df['desc_clean'] = df['product_description'].str.lower().fillna('none').apply(
    lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['desc_clean'] = df['desc_clean'].apply(
    lambda x: x.strip(string.punctuation))
df['desc_clean'] = df['desc_clean'].str.replace('\d+', '')

In [4]:
lemmat = WordNetLemmatizer()
df['prod_lemat'] = df.apply(lambda row: lemmat.lemmatize(row['prod_clean']),
                            axis=1)
df['desc_lemat'] = df.apply(lambda row: lemmat.lemmatize(row['desc_clean']),
                            axis=1)

In [5]:
df['prod_token'] = df.apply(lambda row: word_tokenize(row['prod_lemat']),
                            axis=1)
df['desc_token'] = df.apply(lambda row: word_tokenize(row['desc_lemat']),
                            axis=1)

In [6]:
df['join_text'] = df['prod_token'] + df['desc_token']

In [7]:
df.head()

,id,query,product_title,product_description,prod_clean,desc_clean,prod_lemat,desc_lemat,prod_token,desc_token,join_text
0,3,electric griddle,Star-Max 48 in Electric Griddle,NaN,star-max electric griddle,none,star-max electric griddle,none,"[star-max, electric, griddle]",[none],"[star-max, electric, griddle, none]"
1,6,phillips coffee maker,Philips SENSEO HD7810 WHITE Single Serve Pod C...,NaN,philips senseo hd white single serve pod coffe...,none,philips senseo hd white single serve pod coffe...,none,"[philips, senseo, hd, white, single, serve, po...",[none],"[philips, senseo, hd, white, single, serve, po..."
2,9,san francisco 49ers,2013 San Francisco 49ers Clock,A 2013 San Francisco 49ers clock is the ultima...,san francisco ers clock,san francisco ers clock ultimate way show tea...,san francisco ers clock,san francisco ers clock ultimate way show tea...,"[san, francisco, ers, clock]","[san, francisco, ers, clock, ultimate, way, sh...","[san, francisco, ers, clock, san, francisco, e..."
3,11,aveeno shampoo,AVEENO 10.5FLOZ NRSH SHINE SH,"Water, Ammonium Lauryl Sulfate, Dimethicone, S...",aveeno .floz nrsh shine sh,"water, ammonium lauryl sulfate, dimethicone, s...",aveeno .floz nrsh shine sh,"water, ammonium lauryl sulfate, dimethicone, s...","[aveeno, .floz, nrsh, shine, sh]","[water, ,, ammonium, lauryl, sulfate, ,, dimet...","[aveeno, .floz, nrsh, shine, sh, water, ,, amm..."
4,12,flea and tick control for dogs,Merial Frontline Plus Flea and Tick Control fo...,NaN,merial frontline plus flea tick control dogs p...,none,merial frontline plus flea tick control dogs p...,none,"[merial, frontline, plus, flea, tick, control,...",[none],"[merial, frontline, plus, flea, tick, control,..."


In [8]:
df['text'] = df['join_text'].apply(lambda x: ' '.join(map(str, x)))

In [9]:
df['text_fin'] = (df['text'].str.split().apply(
    lambda x: OrderedDict.fromkeys(x).keys()).str.join(' '))

In [10]:
df_select = df[['id', 'query', 'text_fin']]

In [11]:
df_select.head()

,id,query,text_fin
0,3,electric griddle,star-max electric griddle none
1,6,phillips coffee maker,philips senseo hd white single serve pod coffe...
2,9,san francisco 49ers,san francisco ers clock ultimate way show team...
3,11,aveeno shampoo,"aveeno .floz nrsh shine sh water , ammonium la..."
4,12,flea and tick control for dogs,merial frontline plus flea tick control dogs p...


In [12]:
nlp = spacy.load('en_core_web_md')

C:\Users\sit_k\anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_md' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [13]:
df_select['q_nlp'] = df_select['query'].swifter.apply(nlp)

<ipython-input-13-191f8010f1c8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['q_nlp'] = df_select['query'].swifter.apply(nlp)


In [14]:
df_select['t_nlp'] = df_select['text_fin'].swifter.apply(nlp)

<ipython-input-14-6e9b20057278>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['t_nlp'] = df_select['text_fin'].swifter.apply(nlp)


In [15]:
df_select['sim'] = [
    df_select['q_nlp'][i].similarity(df_select['t_nlp'][i])
    for i in range(len(df_select))
]

<ipython-input-15-f5b4146938fb>:2: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  df_select['q_nlp'][i].similarity(df_select['t_nlp'][i])
<ipython-input-15-f5b4146938fb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['sim'] = [


In [16]:
df_select['fuzzy'] = [
    fuzz.partial_ratio(df_select['query'][x], df_select['text_fin'][x])
    for x in range(len(df_select))
]

<ipython-input-16-fa7327db0a7f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['fuzzy'] = [


In [17]:
df_select['query_len'] = df_select['query'].str.count(' ') + 1

<ipython-input-17-230b04f104ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['query_len'] = df_select['query'].str.count(' ') + 1


In [18]:
tup_list = pickle.load(open('data/tup_list.pkl', "rb"))

In [19]:
for term in tup_list:
    df_select[term] = df_select['query'].str.contains(term)

<ipython-input-19-9fd8d8d75f02>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select[term] = df_select['query'].str.contains(term)


In [20]:
df_select.set_index('id').to_csv('./data/df_test.csv')